In [1]:
from model_MultiResUnet import *
from data import *

import os, os.path

vid="v016_5" #version id should match the file number. Last number shows the cross-validation fold number
# Ran on DellWS with GeForce RTX3060 GPU

# Count the number of train and valid files
train_dir = 'mass_seg_08/train0'+vid[-1]+'/mg'
train_count=len([name for name in os.listdir(train_dir) if os.path.isfile(os.path.join(train_dir, name))])

valid_dir = 'mass_seg_08/valid0'+vid[-1]+'/mg'
valid_count=len([name for name in os.listdir(valid_dir) if os.path.isfile(os.path.join(valid_dir, name))])

### Train with data generator

In [2]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger

#Data augmentation
data_gen_args = dict(rotation_range=90,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.2,
                    zoom_range=0.2,
                    #brightness_range=[0.95,1.05],
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='wrap')

# SETTINGS ***
batch_size=2
learning_rate=1e-4

train_gen = trainGenerator(batch_size,'mass_seg_08/train0'+vid[-1],'mg','mask',data_gen_args,save_to_dir = None)
valid_gen = trainGenerator(batch_size,'mass_seg_08/valid0'+vid[-1],'mg','mask',data_gen_args,save_to_dir = None)

# train_count images are used for training, valid_count images for validating
train_steps = train_count//batch_size
valid_steps = valid_count//batch_size

# SETTINGS ***
loss=dice_loss
steps_per_epoch=3*train_steps
num_epochs=100

model = build_multiresunet((640,640,3))

opt = tf.keras.optimizers.Adam(learning_rate)
metrics = ["acc", dice_coef, iou]
model.compile(loss=loss, optimizer=opt, metrics=metrics)

callbacks = [
             ModelCheckpoint('files_mass_seg_xval/unet_mass_seg_'+vid+'.hdf5', verbose=1, save_best_model=True),
             ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.1, verbose=1, min_lr=1e-8),
             CSVLogger("files_mass_seg_xval/data_"+vid+".csv"),
             EarlyStopping(monitor="val_loss", patience=5, verbose=1)
            ]

model.fit(train_gen, validation_data=valid_gen, steps_per_epoch=steps_per_epoch, validation_steps=valid_steps, 
                    epochs=num_epochs, callbacks=callbacks)


Found 283 images belonging to 1 classes.
Found 283 images belonging to 1 classes.
Epoch 1/100
423/423 [==============================] - ETA: 0s - loss: 0.8914 - acc: 0.8108 - dice_coef: 0.1086 - iou: 0.0627Found 69 images belonging to 1 classes.
Found 69 images belonging to 1 classes.

Epoch 1: saving model to files_mass_seg_xval\unet_mass_seg_v016_5.hdf5
423/423 [==============================] - 398s 874ms/step - loss: 0.8914 - acc: 0.8108 - dice_coef: 0.1086 - iou: 0.0627 - val_loss: 0.8952 - val_acc: 0.8854 - val_dice_coef: 0.1048 - val_iou: 0.0576 - lr: 1.0000e-04
Epoch 2/100
423/423 [==============================] - ETA: 0s - loss: 0.8458 - acc: 0.8987 - dice_coef: 0.1540 - iou: 0.0900
Epoch 2: saving model to files_mass_seg_xval\unet_mass_seg_v016_5.hdf5
423/423 [==============================] - 364s 860ms/step - loss: 0.8458 - acc: 0.8987 - dice_coef: 0.1540 - iou: 0.0900 - val_loss: 0.7839 - val_acc: 0.9332 - val_dice_coef: 0.2161 - val_iou: 0.1312 - lr: 1.0000e-04
Epoch 3/

423/423 [==============================] - ETA: 0s - loss: 0.5169 - acc: 0.9843 - dice_coef: 0.4830 - iou: 0.3549
Epoch 20: saving model to files_mass_seg_xval\unet_mass_seg_v016_5.hdf5
423/423 [==============================] - 365s 862ms/step - loss: 0.5169 - acc: 0.9843 - dice_coef: 0.4830 - iou: 0.3549 - val_loss: 0.6231 - val_acc: 0.9842 - val_dice_coef: 0.3769 - val_iou: 0.2757 - lr: 1.0000e-05
Epoch 21/100
423/423 [==============================] - ETA: 0s - loss: 0.4927 - acc: 0.9842 - dice_coef: 0.5078 - iou: 0.3789
Epoch 21: saving model to files_mass_seg_xval\unet_mass_seg_v016_5.hdf5
423/423 [==============================] - 364s 859ms/step - loss: 0.4927 - acc: 0.9842 - dice_coef: 0.5078 - iou: 0.3789 - val_loss: 0.6288 - val_acc: 0.9846 - val_dice_coef: 0.3712 - val_iou: 0.2713 - lr: 1.0000e-05
Epoch 21: early stopping


### validate your model and save predicted results

In [3]:
validGene = testGenerator("mass_seg_08/valid0"+vid[-1]+"/pred",num_image=valid_count)
model.load_weights("files_mass_seg_xval/unet_mass_seg_"+vid+".hdf5")
results = model.predict(validGene,valid_count,verbose=1)
saveResult("mass_seg_08/valid0"+vid[-1]+"/pred",results,vid[:4])

69/69 [==============================] - 19s 251ms/step


D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_08/valid05/pred\1_predict_v016.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_08/valid05/pred\4_predict_v016.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_08/valid05/pred\5_predict_v016.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_08/valid05/pred\6_predict_v016.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_08/valid05/pred\9_predict_v016.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"

In [4]:
model.summary()

Model: "MultiResUNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 640, 640, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 640, 640, 8)  216         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 640, 640, 8)  32         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                       

 concatenate_1 (Concatenate)    (None, 320, 320, 10  0           ['activation_8[0][0]',           
                                5)                                'activation_9[0][0]',           
                                                                  'activation_10[0][0]']          
                                                                                                  
 conv2d_15 (Conv2D)             (None, 320, 320, 10  5355        ['max_pooling2d[0][0]']          
                                5)                                                                
                                                                                                  
 batch_normalization_21 (BatchN  (None, 320, 320, 10  420        ['concatenate_1[0][0]']          
 ormalization)                  5)                                                                
                                                                                                  
 batch_nor

 conv2d_31 (Conv2D)             (None, 80, 80, 142)  90738       ['activation_21[0][0]']          
                                                                                                  
 batch_normalization_46 (BatchN  (None, 80, 80, 142)  568        ['conv2d_31[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_22 (Activation)     (None, 80, 80, 142)  0           ['batch_normalization_46[0][0]'] 
                                                                                                  
 conv2d_32 (Conv2D)             (None, 80, 80, 213)  272214      ['activation_22[0][0]']          
                                                                                                  
 batch_normalization_47 (BatchN  (None, 80, 80, 213)  852        ['conv2d_32[0][0]']              
 ormalizat

                                                                                                  
 tf.__operators__.add_13 (TFOpL  (None, 80, 80, 256)  0          ['batch_normalization_51[0][0]', 
 ambda)                                                           'batch_normalization_52[0][0]'] 
                                                                                                  
 batch_normalization_59 (BatchN  (None, 40, 40, 853)  3412       ['activation_29[0][0]']          
 ormalization)                                                                                    
                                                                                                  
 activation_25 (Activation)     (None, 80, 80, 256)  0           ['tf.__operators__.add_13[0][0]']
                                                                                                  
 conv2d_transpose (Conv2DTransp  (None, 80, 80, 256)  873728     ['batch_normalization_59[0][0]'] 
 ose)     

                                                                                                  
 batch_normalization_43 (BatchN  (None, 160, 160, 12  512        ['conv2d_29[0][0]']              
 ormalization)                  8)                                                                
                                                                                                  
 activation_33 (Activation)     (None, 80, 80, 426)  0           ['tf.__operators__.add_15[0][0]']
                                                                                                  
 tf.__operators__.add_11 (TFOpL  (None, 160, 160, 12  0          ['batch_normalization_42[0][0]', 
 ambda)                         8)                                'batch_normalization_43[0][0]'] 
                                                                                                  
 batch_normalization_65 (BatchN  (None, 80, 80, 426)  1704       ['activation_33[0][0]']          
 ormalizat

                                                                                                  
 activation_36 (Activation)     (None, 160, 160, 10  0           ['batch_normalization_68[0][0]'] 
                                6)                                                                
                                                                                                  
 activation_13 (Activation)     (None, 320, 320, 64  0           ['tf.__operators__.add_7[0][0]'] 
                                )                                                                 
                                                                                                  
 concatenate_8 (Concatenate)    (None, 160, 160, 21  0           ['activation_34[0][0]',          
                                2)                                'activation_35[0][0]',          
                                                                  'activation_36[0][0]']          
          

 conv2d_6 (Conv2D)              (None, 640, 640, 32  9216        ['batch_normalization_8[0][0]']  
                                )                                                                 
                                                                                                  
 conv2d_7 (Conv2D)              (None, 640, 640, 32  1024        ['batch_normalization_8[0][0]']  
                                )                                                                 
                                                                                                  
 batch_normalization_72 (BatchN  (None, 320, 320, 17  68         ['conv2d_48[0][0]']              
 ormalization)                  )                                                                 
                                                                                                  
 batch_normalization_9 (BatchNo  (None, 640, 640, 32  128        ['conv2d_6[0][0]']               
 rmalizati

                                )                                                                 
                                                                                                  
 tf.__operators__.add_17 (TFOpL  (None, 320, 320, 10  0          ['batch_normalization_75[0][0]', 
 ambda)                         5)                                'batch_normalization_76[0][0]'] 
                                                                                                  
 batch_normalization_15 (BatchN  (None, 640, 640, 32  128        ['conv2d_10[0][0]']              
 ormalization)                  )                                                                 
                                                                                                  
 batch_normalization_16 (BatchN  (None, 640, 640, 32  128        ['conv2d_11[0][0]']              
 ormalization)                  )                                                                 
          

__________________________________________________________________________________________________
